# M161 first question notebook- 


In [18]:
import joblib

dataTrain = joblib.load(r'joblibCache\dataTrain_cleaned.joblib')
dataTrain.head()

,Id,Title,Content,Label
0,227464,come cabl groceri overlord,subscrib one three dink compar speak cabl abl ...,Entertainment
1,244074,presid react happi,presid react happi singer presid took twitter ...,Entertainment
2,60707,wildlif servic,fish wildlif servic comment period addit day p...,Technology
3,27883,launch,natur social medium often sourc real time brea...,Technology
4,169596,u new york casino,u new york casino latest news top deck world e...,Business


In [19]:
# # Stratify and keep 50000 instances based on the 'Label' column
# from sklearn.model_selection import train_test_split

# # Stratify and sample 50000 instances
# stratified_data, _ = train_test_split(
#     dataTrain,
#     train_size=50000,
#     stratify=dataTrain['Label'],
#     random_state=42
# )
# dataTrain = stratified_data.reset_index(drop=True)
# print(f"Subset shape (stratified): {dataTrain.shape}")

### Just printing out the firtst 5 columns to see what happend to text

In [20]:
print(dataTrain.info())

<class 'pandas.DataFrame'>
RangeIndex: 111220 entries, 0 to 111219
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   Id       111220 non-null  int64
 1   Title    111220 non-null  str  
 2   Content  111220 non-null  str  
 3   Label    111220 non-null  str  
dtypes: int64(1), str(3)
memory usage: 3.4 MB
None


In [ ]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# 1. PREPARE DATA FOR WORD2VEC
# Word2Vec requires a list of tokens (words), not just raw strings.
# Since your data looks already cleaned, we can just split by whitespace.
tokenized_content = dataTrain['Content'].apply(lambda x: x.split())

# 2. TRAIN WORD2VEC MODEL
# vector_size=100: Dimension of the dense vector
# window=5: Distance between current and predicted word
# min_count=2: Ignores all words with total frequency lower than this
# workers=4: Number of threads to use
w2v_model = Word2Vec(sentences=tokenized_content, vector_size=300, window=5, min_count=2, workers=4)

# 3. VECTORIZE DOCUMENTS (AVERAGE WORD VECTORS)
# We need to turn each document into a single vector to feed into the MLP.
# A common strategy is to average the vectors of all words in the document.

def document_vectorizer(tokens, model):
    # Filter out words that are not in the Word2Vec vocabulary
    valid_words = [word for word in tokens if word in model.wv.key_to_index]
    
    if valid_words:
        # Calculate the mean of vectors for valid words
        return np.mean(model.wv[valid_words], axis=0)
    else:
        # Return a vector of zeros if no words are found in the vocab
        return np.zeros(model.vector_size)

# Apply the vectorizer to create the input feature matrix X
X = np.array([document_vectorizer(tokens, w2v_model) for tokens in tokenized_content])
y = dataTrain['Label']

# 4. SPLIT DATA
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 5. TRAIN MLP CLASSIFIER
# hidden_layer_sizes=(100, 50): Two hidden layers with 100 and 50 neurons
# max_iter=300: Maximum number of iterations (epochs)
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=300, activation='relu', solver='adam', random_state=42)

print("Training MLP Classifier...")
mlp.fit(X_train, y_train)

# 6. EVALUATE
y_pred = mlp.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

Training MLP Classifier...
Accuracy: 0.9446592339507283

Classification Report:

               precision    recall  f1-score   support

     Business       0.92      0.89      0.91      4948
Entertainment       0.98      0.98      0.98      8905
       Health       0.94      0.95      0.95      2391
   Technology       0.91      0.94      0.93      6000

     accuracy                           0.94     22244
    macro avg       0.94      0.94      0.94     22244
 weighted avg       0.94      0.94      0.94     22244



In [22]:
# 5. TRAIN MLP CLASSIFIER
# hidden_layer_sizes=(100, 50): Two hidden layers with 100 and 50 neurons
# max_iter=300: Maximum number of iterations (epochs)
mlp = MLPClassifier(hidden_layer_sizes=(500, 250), max_iter=300, activation='relu', solver='adam', random_state=42)

print("Training MLP Classifier...")
mlp.fit(X_train, y_train)

# 6. EVALUATE
y_pred = mlp.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

Training MLP Classifier...
Accuracy: 0.9540999820176227

Classification Report:

               precision    recall  f1-score   support

     Business       0.93      0.92      0.92      4948
Entertainment       0.98      0.98      0.98      8905
       Health       0.96      0.96      0.96      2391
   Technology       0.93      0.94      0.94      6000

     accuracy                           0.95     22244
    macro avg       0.95      0.95      0.95     22244
 weighted avg       0.95      0.95      0.95     22244



In [23]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# 1. PREPARE DATA FOR WORD2VEC
# Word2Vec requires a list of tokens (words), not just raw strings.
# Since your data looks already cleaned, we can just split by whitespace.
tokenized_content = dataTrain['Content'].apply(lambda x: x.split())

# 2. TRAIN WORD2VEC MODEL
# vector_size=100: Dimension of the dense vector
# window=5: Distance between current and predicted word
# min_count=2: Ignores all words with total frequency lower than this
# workers=4: Number of threads to use
w2v_model = Word2Vec(sentences=tokenized_content, vector_size=1000, window=5, min_count=2, workers=4)

# 3. VECTORIZE DOCUMENTS (AVERAGE WORD VECTORS)
# We need to turn each document into a single vector to feed into the MLP.
# A common strategy is to average the vectors of all words in the document.

def document_vectorizer(tokens, model):
    # Filter out words that are not in the Word2Vec vocabulary
    valid_words = [word for word in tokens if word in model.wv.key_to_index]
    
    if valid_words:
        # Calculate the mean of vectors for valid words
        return np.mean(model.wv[valid_words], axis=0)
    else:
        # Return a vector of zeros if no words are found in the vocab
        return np.zeros(model.vector_size)

# Apply the vectorizer to create the input feature matrix X
X = np.array([document_vectorizer(tokens, w2v_model) for tokens in tokenized_content])
y = dataTrain['Label']

# 4. SPLIT DATA
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 5. TRAIN MLP CLASSIFIER
# hidden_layer_sizes=(100, 50): Two hidden layers with 100 and 50 neurons
# max_iter=300: Maximum number of iterations (epochs)
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=300, activation='relu', solver='adam', random_state=42)

print("Training MLP Classifier...")
mlp.fit(X_train, y_train)

# 6. EVALUATE
y_pred = mlp.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

Training MLP Classifier...
Accuracy: 0.9503686387340406

Classification Report:

               precision    recall  f1-score   support

     Business       0.93      0.91      0.92      4948
Entertainment       0.98      0.98      0.98      8905
       Health       0.95      0.96      0.95      2391
   Technology       0.93      0.94      0.93      6000

     accuracy                           0.95     22244
    macro avg       0.95      0.95      0.95     22244
 weighted avg       0.95      0.95      0.95     22244

